In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance
from scipy.sparse.csgraph import minimum_spanning_tree
import math


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from modules.dp_bounds import *



In [3]:
# Set the mean and covariance matrix for each Gaussian distribution


mean1 = [0, 0, 0 , 0 , 0, 0, 0 , 0]
# covariance1 = np.ones((3,3))
covariance1 = np.identity(8)
# covariance1 = [[1, 0, 0],  [0, 1, 0], [0, 0, 0]

mean2 = [2.56, 0, 0 , 0, 0, 0,0, 0 ]
# mean2 = [math.sqrt(1/2), math.sqrt(1/2), 0]

# covariance2 = np.ones((3,3))
covariance2= np.identity(8)

#choose the number of samples per group. 
samples = 500

n0 = samples
n1 = samples

# p, q = .5, .5

In [4]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [5]:
# def get_FR(data1, data2):
    
#     dataset = np.concatenate([data1, data2])

#     FR_statistic =  0 

#     # Calculate pairwise distances
#     distances = distance.pdist(dataset)

#     # Create a square distance matrix
#     dist_matrix = distance.squareform(distances)

#     # Create a minimum spanning tree
#     mst = minimum_spanning_tree(dist_matrix)

#     # Extract edges from the minimum spanning tree
#     edges = np.array(np.where(mst.toarray() > 0)).T


#     for edge in edges:
#         if dataset[edge[0]] in data1 and dataset[edge[1]] in data1:
#             continue
#         elif dataset[edge[0]] in data2 and dataset[edge[1]] in data2:
#             continue
#         else:
#             FR_statistic  +=1
            
#     return FR_statistic

In [6]:
# def get_bounds(up):
# #     print(up)
        
#     lower = 1/2 - 1/2 *math.sqrt(up) 
#     upper = 1/2 - 1/2 * up 
#     return lower, upper 

# #     print("Baye's error rate is between", 1/2 - val1, 1/2 -  1/2 * up )

In [7]:
def analyze(dataset):
    FR_statistic =  0 

    # Calculate pairwise distances

    distances = distance.pdist(dataset)

    # Create a square distance matrix
    dist_matrix = distance.squareform(distances)

    # Create a minimum spanning tree
    mst = minimum_spanning_tree(dist_matrix)

    # Extract edges from the minimum spanning tree
    edges = np.array(np.where(mst.toarray() > 0)).T

    # Plot the dataset and the minimum spanning tree
    fig = plt.figure(figsize = (7,11), )
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(dataset[:, 0], dataset[:, 1], dataset[:, 2], c='black', marker='o')

    for edge in edges:
        if dataset[edge[0]] in data1 and dataset[edge[1]] in data1:
            color = 'blue'
#             marker = 'blue'
        elif dataset[edge[0]] in data2 and dataset[edge[1]] in data2:
            color = 'red'
#             marker = "r"
        else:
            color = 'cyan'
#             marker = 'g'
            FR_statistic  +=1

        ax.plot(dataset[edge, 0], dataset[edge, 1], dataset[edge, 2], c=color, )

    plt.show()
    print("Frieman-Rafsky statistic = ",  FR_statistic )

In [8]:

bounds_l = []
bounds_u = []

MC_num = 500

for i in range(MC_num):
    # Generate random samples from the multivariate normal distributions
    data1 = np.random.multivariate_normal(mean1, covariance1, n0)
    data2 = np.random.multivariate_normal(mean2, covariance2, n1)

    
    FR = get_FR(data1, data2) #get the Frieman Rafsky statistic. 

    Dp = 1 - FR * (n0 + n1)/ (2 * n0 * n1)
    
    Up = Dp # we have choosen p =q
    if Up <= 0:
        print(i, FR, Up)
        analyze( np.concatenate([data1, data2]))
        
        
    lower, upper = get_bounds(Up)
    
#     print("1, ", 1/2 - 1/2 * math.sqrt(Dp)  , FR/ (n0+ n1))
#     print("2, ", lower, upper)
    
    bounds_l.append(lower)
    bounds_u.append(upper)
    

print("The Bounds")

lower = np.average(bounds_l)
upper = np.average(bounds_u)
print(lower)
print(upper)

The Bounds
0.09359974293584655
0.16952


In [9]:
print(min(bounds_l))
print(max(bounds_l))


print(min(bounds_u))
print(max(bounds_u))

0.06872282694304349
0.12250827823646249
0.128
0.21499999999999997


In [15]:
from scipy import stats
ub = stats.describe(bounds_u)

In [22]:
ub

DescribeResult(nobs=500, minmax=(0.128, 0.21499999999999997), mean=0.16952, variance=0.00020796553106212391, skewness=0.1198449474679653, kurtosis=-0.20373387189369385)

In [11]:
dx = 0.001
x = np.arange(-5, 5 + dx, dx)

f0 = np.exp(-0.5 * (x - 0)**2) / np.sqrt(2 * np.pi)
f1 = np.exp(-0.5 * (x - 1)**2) / np.sqrt(2 * np.pi)

fmin = np.minimum(f0, f1)

BER = 0.5 * np.sum(fmin * dx)

print("BER 2d:", BER)

BER 2d: 0.30853741068873275


In [12]:
BER_lst = np.ones(MC_num) * BER


mse_l = ((BER_lst - bounds_l)**2).mean(axis=0)

mse_u =  ((BER_lst - bounds_u)**2).mean(axis=0)

print(mse_l, mse_u)

0.04627703207724008 0.019533390074599782
